# Open-shell

## Currently, open-shell calculations support only linear mixing in SCF optimization.
## Can do geometry optimization but not MD.
## Will be fixed soon.

In [2]:
import torch
import seqm
from seqm.seqm_functions.constants import Constants
from seqm.seqm_functions.make_dm_guess import make_dm_guess
import matplotlib.pyplot as plt

from seqm.Molecule import Molecule
from seqm.ElectronicStructure import Electronic_Structure
import numpy as np
from seqm.seqm_functions.read_xyz import read_xyz
from seqm.MolecularDynamics import Geometry_Optimization_SD

DTYPE = torch.float64
torch.set_default_dtype(DTYPE)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Carbene (CH2) and O2, triplets

In [6]:
species = torch.as_tensor([
                            [6,1,1],
                            [8,8,0]
                          ], dtype=torch.int64, device=device)

coordinates = torch.tensor([
                            [[ 1.2437e-01,  4.4651e-02,  1.7825e-01],
                             [ 1.1301e+00, -3.9041e-02, -1.5585e-01],
                             [-5.6454e-01,  2.4138e-01,  9.6361e-01]],

                            [[ 1.9663e-04,  3.0676e-04, -1.0761e-03],
                             [ 3.0337e-04,  9.3247e-05,  1.1676e+00],
                             [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]
    
                            ], device=device)

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))

seqm_parameters = {
                   'method' : 'PM3',
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.5], # converger used for scf loop
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                   'scf_backward' : 0,
                   'UHF' : True, ### use unrestricted HF
                   }

# specify charges and multiplicity
charges = torch.tensor([0,0],dtype=torch.int64, device=device)
mult = torch.tensor([3,3], device=device)

molecule = Molecule(const, seqm_parameters, coordinates, species, charges, mult).to(device)

In [8]:
# optimize geometry
opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.01, force_tol=1.0e-2, max_evl=400).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      8.860222e-03 ||3.106364e+00 3.106364e+00 ||-1.812879e-01 -1.812879e-01 
converged with 1 step, Max Force = 8.860222e-03 (eV/Ang), dE = 1.462538e+00 (eV)


In [9]:
# Total energy
molecule.Etot

tensor([-146.1992, -584.0310], device='cuda:0')

## Compare with energies of singlets

In [10]:
coordinates = torch.tensor([
                            [[-2.2974e-02, -7.9243e-03, -3.1638e-02],
                             [ 1.0669e+00,  9.8889e-03,  3.9484e-02],
                             [-3.5404e-01,  2.4503e-01,  9.7816e-01]],

                            [[ 1.9159e-04,  3.1941e-04, -1.4933e-03],
                             [ 3.0841e-04,  8.0601e-05,  1.1680e+00],
                             [ 0.0000e+00,  0.0000e+00,  0.0000e+00]]
    
                            ], device=device)

charges = torch.tensor([0,0],dtype=torch.int64, device=device)
mult = torch.tensor([1,1], device=device)

molecule = Molecule(const, seqm_parameters, coordinates, species, charges, mult).to(device)


In [11]:
opt =  Geometry_Optimization_SD(seqm_parameters, alpha=0.01, force_tol=1.0e-2, max_evl=400).to(device)
max_force, dE =  opt.run(molecule)

Step,  Max_Force,      Etot(eV),     dE(eV)
1      9.515048e-03 ||4.909879e+00 4.909879e+00 ||7.971812e-01 7.971812e-01 
converged with 1 step, Max Force = 9.515048e-03 (eV/Ang), dE = 2.853530e+00 (eV)


In [12]:
# Energies of singlets are higher than energies of triplets, as expected
molecule.Etot

tensor([-144.3957, -583.0526], device='cuda:0')

# Open-shell singlet
### H2 and HF separation
### The symmetry needs to be broken. Otherwise, closed-shell solution.
### Calculations will take a while because of linear mixing which is not the most efficient SCF optimization algorythm. Adaptive mixing for open shells will be added soon.

In [13]:
# array of bond lengths
bond_l = np.linspace(0.7, 10.0, 41)

species = torch.as_tensor([
                       [1,1],
                       [9,1],
                            ],
                      dtype=torch.int64, device=device)
const = Constants().to(device)
elements = [0]+sorted(set(species.reshape(-1).tolist()))
charges = torch.tensor([0,0],dtype=torch.int64, device=device)

### Broken symmetry open-shell.

In [ ]:
# arrays to record energies and charges
E_os_break = np.array([[0,0]])
Q_os_break = np.array([[[0,0],[0,0]]])

for i in bond_l:
    coordinates = torch.tensor([
                              [
                               [ 0.00,    0.00,    0.00],
                               [ 0.00,    0.00,    i],
                              ],
                              [
                               [0.00,    0.00,    0.00],
                               [0.00,    0.00,    i],
                              ],
                            ], device=device)
    
    seqm_parameters = {
                   'method' : 'PM3',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-7,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.95], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                   'scf_backward' : 0,
                   'UHF' : True, # use unrestricted HF
                   }
    
    mult = torch.tensor([1,1], device=device)
    molecule = Molecule(const, seqm_parameters, coordinates, species, charges, mult).to(device)
    
    ### Break the symmetry of a DM guess: HOMO_alpha = 0.9*HOMO + 0.1*LUMO   (mix_coeff=0.1)
    make_dm_guess(molecule, seqm_parameters, mix_homo_lumo=True, mix_coeff=0.1, overwrite_existing_dm=True);
    
    # Single point calculation with the current bond length
    esdriver = Electronic_Structure(seqm_parameters).to(device)
    esdriver(molecule, P0=molecule.dm)
    
    E_os_break = np.vstack((E_os_break,molecule.Etot.cpu().numpy()))
    Q_os_break = np.vstack((Q_os_break,[molecule.q.cpu().numpy()]))
    
E_os_break = E_os_break[1:]
Q_os_break = Q_os_break[1:]

Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM
Reinitializing DM


###  Closed-shell (equivalent to open-shell without breaking the symetry).

In [ ]:
E_cs = np.array([[0,0]])
Q_cs = np.array([[[0,0],[0,0]]])

dm=None
for i in bond_l:

    coordinates = torch.tensor([
                              [[ 0.00,    0.00,    0.00],
                               [ 0.00,    0.00,    i]],
        
                              [[0.00,    0.00,    0.00],
                               [0.00,    0.00,    i]],
                            ], device=device)
    
    
    seqm_parameters = {
                   'method' : 'PM3',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [0,0.95], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True,
                   'scf_backward' : 0,
                   'UHF' : False, # use unrestricted HF
                   }
    
    mult = 1
    molecule = Molecule(const, seqm_parameters, coordinates, species, charges, mult).to(device)
    esdriver = Electronic_Structure(seqm_parameters).to(device)
    esdriver(molecule)
    E_cs = np.vstack((E_cs,molecule.Etot.cpu().numpy()))
    Q_cs = np.vstack((Q_cs,[molecule.q.cpu().numpy()]))

E_cs = E_cs[1:]
Q_cs = Q_cs[1:]

### H2 separation energy curve.

In [ ]:
plt.plot(bond_l, E_cs[:,0] - min(E_cs[:,0]), label='RHF')
plt.plot(bond_l, E_os_break[:,0] - min(E_os_break[:,0]), label='UHF')
plt.legend()

### HF separation energy curve.

In [ ]:
plt.plot(bond_l, E_cs[:,1]-min(E_cs[:,1]), label='RHF')
plt.plot(bond_l, E_os_break[:,1] - min(E_os_break[:,1]), label='UHF')
#plt.plot(E_os_break_reuse_dm[:,1])
plt.legend()

### Charges on F and H at 0.93 Å (equilibrium) distance.

#### Broken symmetry open-shell singlet

In [ ]:
# F                    H
Q_os_break[:,1,0][1], Q_os_break[:,1,1][1]

#### Closed-shell singlet

In [ ]:
# F                    H
Q_cs[:,1,0][1], Q_cs[:,1,1][1]

### Charges on F and H at 10 Å distance.

#### Broken symmetry open-shell singlet. Almost zero (homolytic separation), which is correct.

In [ ]:
# F                    H
Q_os_break[:,1,0][-1], Q_os_break[:,1,1][-1]

#### Closed-shell singlet. Non-zero charges, which is incorrect. In gas phase, separation shoud proceed homolytically.

In [ ]:
# F                    H
Q_cs[:,1,0][-1], Q_cs[:,1,1][-1]